# Clasificación supervisada


El reto que se va a resolver, es la clasificación de reseñas de hoteles de Andalucía en el sentimiento asociado a la visita. Los textos se encuentran en el dataset `andalucia_hoteles.csv`en la columna `review_text`, y el sentimiento en la columna `label`.

Considera también detectar el idioma y filtrar por aquellas reviews que sean en español

La clasificación supervisada en textos funciona conceptualmente de manera similar a la clasificación en otros problemas de Machine Learning con datos estructurados:

1. Se requiere preprocesar la información (en el caso de datos no estructurados, convertir los textos a TFIDF).
2. Dividir en entrenamiento y test el conjunto de textos.
3. Entrenar al modelo incluyendo el set de train.
4. Evaluación del modelo, lanzando la predicción sobre el conjunto de test y evaluándolo con los valores reales.
 Puedes hacerlo en notebooks diferentes (cada uno de los modelos) o todos en el mismo. Sigue la secuencia de pasos anterior, aplicando correctamente las funciones necesarias en cada paso, para cada uno de los modelos:

- Clasificador ingenuo bayesiano
- SVM
- KNN
- Decision tree
- Random Forest
- GBT

¿Cuál funciona mejor? ¿En qué métricas te has basado?

In [1]:
## Importación de librerías

import spacy
import pandas as pd

nlp_español = spacy.load('es_core_news_lg')  

/Users/irene/opt/anaconda3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-16 12:00:25.606567: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
## Lectura de datos

datos = pd.read_csv('data/andalucia_hoteles.csv')
print(datos.head())
print(datos.shape)
print(datos.columns)

   Unnamed: 0                                 title  rating  \
0           0                   IMPOSIBLE DESCANSAR       2   
1           1          No es oro todo lo que reluce       3   
2           2        Un buen hotel con mucho ruido.       3   
3           3  SIN CALEFACCIÓN OPERATIVA Y CON FRÍO       2   
4           4             Deja bastante que desear.       2   

                                         review_text  \
0  El fin de semana mi pareja y yo hicimos una re...   
1  El hotel en general está bien, las habtiacione...   
2  El hotel es moderno, amplio y limpio, pero no ...   
3  Calefacción averiada o no operativa. Se coment...   
4  Este hotel ha bajado notoriamente su categoria...   

                                location                hotel  label  
0  Seville_Province_of_Seville_Andalucia  Hotel_Rey_Alfonso_X      0  
1  Seville_Province_of_Seville_Andalucia  Hotel_Rey_Alfonso_X      3  
2  Seville_Province_of_Seville_Andalucia  Hotel_Rey_Alfonso_X      3  


### 1.1. Preprocesamiento y normalización
Vamos a separar los documentos y sus categorías. docs y categs son series de Pandas. Hay que separar las categorías de los documentos para usar estos últimos y obtener la matriz Tf-idf.

In [6]:
docs = datos['review_text'] # extract column with review
categs = datos['label'] # extract column with sentiment

In [7]:
docs

0       El fin de semana mi pareja y yo hicimos una re...
1       El hotel en general está bien, las habtiacione...
2       El hotel es moderno, amplio y limpio, pero no ...
3       Calefacción averiada o no operativa. Se coment...
4       Este hotel ha bajado notoriamente su categoria...
                              ...                        
7610    Bastante cerca del centro de sevilla, servicio...
7611    Séjour touristique à Séville, la découverte de...
7612    Stayed here for 4 nights,  the room was big an...
7613    Séjour touristique à Séville, la découverte de...
7614    Stayed here for 4 nights,  the room was big an...
Name: review_text, Length: 7615, dtype: object

In [8]:
print("Datos es tipo: ", type(datos))
print("Docs es tipo: ", type(docs))
print("Categs es tipo: ", type(categs))

Datos es tipo:  <class 'pandas.core.frame.DataFrame'>
Docs es tipo:  <class 'pandas.core.series.Series'>
Categs es tipo:  <class 'pandas.core.series.Series'>


### 1.2. Obtención de las matrices BOW y Tf-idf

Obten la matriz TFIDF de todos los textos. Se puede obtener a partir de la matriz BOW.

In [9]:
# tokenizamos los documentos y convertimos en matriz BOW

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
docs_bow = vectorizer.fit_transform(docs)


In [10]:
docs[0]

'El fin de semana mi pareja y yo hicimos una reserva en este hotel, con el fin de descansar y desconectar, fue sólo una noche y menos mal.  Nos llevaron a un ala bastante apartada del hotel porque nos dijeron que era mejor para descansar ya que la parte de fuera era muy “jaleosa”. Nos pareció bien porque era justo lo que buscábamos, y cuál fue nuestra sorpresa? Desde las 6 de la mañana con ruidos, primero lo que suponemos que eran unos tacones en la habitación de arriba (de eso no tiene culpa el hotel, obviamente) y después sobre las 7 o poco más, las limpiadoras moviendo muebles y arrastrando sofás o lo que fuera. Habíamos cogido sólo alojamiento para descansar, pensando en no tener que madrugar como habitualmente, pero fue IMPOSIBLE por los ruidos constantes.  Por destacar algo…'

In [11]:
# Construimos la matriz formato Tf-idf

from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer() 
docs_tfidf = tfidf.fit_transform(docs_bow)
docs_tfidf_densa = docs_tfidf.todense()
docs_tfidf_densa

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

### 2. Preparación de los subconjuntos de entrenamiento y test

Divide entre train y test, utilizando train_test_split.

In [12]:
# División mediante train_test_split. Test de 25%

from sklearn.model_selection import train_test_split
docs_train, docs_test, categs_train, categs_test = train_test_split(docs_tfidf, categs, test_size = 0.25, 
                                                                    random_state = 50)

### 3. Entrenamiento del modelo: clasificador ingenuo bayesiano (MultinomialNB)

In [13]:
# Entrenamiento del clasificador NB

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(docs_train, categs_train)

MultinomialNB()

### 4. Evaluación del modelo.

Obtén la confusión matrix para evaluar el rendimiento del modelo, así como el accuracy (utilizando la función score).

In [14]:
# Predicción del set de test

categs_pred = clf.predict(docs_test)

In [15]:
# Confusion Matrix

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(categs_test, categs_pred)
cm

array([[612,   8,  46],
       [  7, 645,  10],
       [157,  62, 357]])

In [16]:
acc_train = clf.score(docs_train, categs_train)
acc_test = clf.score(docs_test, categs_test)

print("Accuracy entrenamiento: ", acc_train)
print("Accuracy PRUEBA: ", acc_test)
print("Fiabilidad: ", acc_test / acc_train)  

Accuracy entrenamiento:  0.9103484503589564
Accuracy PRUEBA:  0.8476890756302521
Fiabilidad:  0.9311699001585632


## SVM

In [17]:
# Entrenamiento del clasificador SVM

from sklearn.svm import SVC
svm = SVC(kernel='linear')
# svm = SVC(kernel='poly')
# svm = SVC(kernel='rbf')
# svm = SVC(kernel='sigmoid')


svm.fit(docs_train, categs_train)

acc_train = svm.score(docs_train, categs_train)
acc_test = svm.score(docs_test, categs_test)

print("Accuracy entrenamiento: ", acc_train)
print("Accuracy PRUEBA: ", acc_test)
print("Fiabilidad: ", acc_test / acc_train)  

Accuracy entrenamiento:  0.9711083873227105
Accuracy PRUEBA:  0.8802521008403361
Fiabilidad:  0.9064406325097656


## KNN

In [ ]:
# Entrenamiento del clasificador KNN

from sklearn.neighbors import KNeighborsClassifier

print("Númro de vecinos cercanos a considerar: ")
num_vecinos = int(input())
knn = KNeighborsClassifier(num_vecinos)

knn.fit(docs_train, categs_train)

acc_train = svm.score(docs_train, categs_train)
acc_test = svm.score(docs_test, categs_test)

print("Accuracy entrenamiento: ", acc_train)
print("Accuracy PRUEBA: ", acc_test)
print("Fiabilidad: ", acc_test / acc_train)  

## Árbol de decisión

In [ ]:
# Entrenamiento del clasificador Árboles de Decisión

from sklearn import tree

a_dec = tree.DecisionTreeClassifier()
a_dec.fit(docs_train, categs_train)

acc_train = a_dec.score(docs_train, categs_train)
acc_test = a_dec.score(docs_test, categs_test)

print("Accuracy entrenamiento: ", acc_train)
print("Accuracy PRUEBA: ", acc_test)
print("Fiabilidad: ", acc_test / acc_train)  